In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

# --- Configuration ---
DATA_PATH = os.path.join('MP_Data') # Path for exported data, numpy arrays
actions = ['hello', 'thanks', 'iloveyou','please','sorry','yes','no'] # Actions that we try to detect
no_sequences = 60 # Number of sequences per action
sequence_length = 30 # Number of frames per sequence (each sequence is 30 frames of data)

# Create directories for data collection
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# --- MediaPipe Setup ---
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    # Draw hand landmarks
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh]) # Combine left and right hand keypoints

# --- Data Collection Loop ---
cap = cv2.VideoCapture(0) # Webcam

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in actions:
        # Loop through sequences (videos)
        for sequence in range(no_sequences):
            # Loop through frames of the video
            for frame_num in range(sequence_length):

                ret, frame = cap.read() # Read feed
                image, results = mediapipe_detection(frame, holistic) # Make detections

                draw_landmarks(image, results) # Draw landmarks

                # Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(100) # Wait 2 seconds
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results) # Export keypoints
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# --- Configuration (match with data collection) ---
DATA_PATH = os.path.join('MP_Data')
actions = ['hello', 'thanks', 'iloveyou','please','sorry','yes','no']
no_sequences = 60
sequence_length = 30

# --- Load Data ---
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# --- Build and Train LSTM Model ---
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126))) # 126 keypoints (2 hands * 21 landmarks * 3 coords)
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax')) # Output layer with softmax for multi-class classification
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback]) # You might need more epochs
model.summary()

# --- Evaluate Model ---
res = model.predict(X_test)
print(f"Prediction for first test sample: {actions[np.argmax(res[0])]}")
print(f"True label for first test sample: {actions[np.argmax(y_test[0])]}")

# --- Save Model ---
model.save('action.h5')
 # Clear the current model from memory if you want to load it later

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time
from tensorflow.keras.models import load_model

# --- Configuration ---
actions = ['hello', 'thanks', 'iloveyou', 'please', 'sorry', 'yes', 'no']
sequence_length = 30

# Higher threshold = only confident predictions will show
threshold = 0.85
# How long the result stays on screen
DISPLAY_DURATION = 2.0
# Number of consistent frames required to confirm gesture
SMOOTHING_WINDOW = 10
# Cooldown after one detection before next one allowed
COOLDOWN_TIME = 3.0

# --- Load Trained Model ---
model = load_model('action.h5')

# --- Mediapipe Setup ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):
    # Draw only visible hands for clarity
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# --- Real-time Detection ---
sequence, predictions = [], []
current_action = "..."
last_confirmed_action = None
display_start_time = 0
last_detection_time = 0

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)

        # Skip prediction if no hands visible
        if not results.left_hand_landmarks and not results.right_hand_landmarks:
            cv2.rectangle(image, (0, 0), (640, 40), (0, 0, 0), -1)
            cv2.putText(image, "Waiting for hand gesture...", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.imshow('Sign Language Detection', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            continue

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]

        if len(sequence) == sequence_length and (time.time() - last_detection_time) > COOLDOWN_TIME:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predicted_action_index = np.argmax(res)
            confidence = res[predicted_action_index]
            action = actions[predicted_action_index] if confidence > threshold else None

            predictions.append(action)
            predictions = predictions[-SMOOTHING_WINDOW:]

            # Confirm stable prediction
            if action and predictions.count(action) == SMOOTHING_WINDOW:
                if action != last_confirmed_action:
                    current_action = action
                    last_confirmed_action = action
                    display_start_time = time.time()
                    last_detection_time = time.time()
                    print(f"✅ Detected: {action}")

        # Reset after display duration
        if time.time() - display_start_time > DISPLAY_DURATION:
            current_action = "..."
            last_confirmed_action = None

        # --- Display ---
        cv2.rectangle(image, (0, 0), (640, 60), (0, 102, 204), -1)
        cv2.putText(image, f"Detected: {current_action}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('Sign Language Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 48ms/step
✅ Detected: please
1/1 [==============================] - 0s 57ms/step
✅ Detected: please
1/1 [==============================] - 0s 48ms/step
✅ Detected: please
1/1 [==============================] - 0s 45ms/step
✅ Detected: please
1/1 [==============================] - 0s 48ms/step
✅ Detected: hello
1/1 [==============================] - 0s 54ms/step
✅ Detected: please
1/1 [==============================] - 0s 59ms/step
✅ Detected: please
1/1 [==============================] - 0s 89ms/step
✅ Detected: hello
1/1 [==============================] - 0s 78ms/step
✅ Detected: hello
1/1 [==============================] - 0s 42ms/step
✅ Detected: hello
1/1 [==============================] - 0s 45ms/step
✅ Detected: please
1/1 [==============================] - 0s 63ms/step
✅ Detected: hello
1/1 [==============================] - 0s 136ms/step
✅ Detected: iloveyou
1/1 [==============================] - 0s 53ms/step
✅ Detected: please
1/1 [===